# Prepare a dataset for ML

- A high quality data is the first step to successful ML
- Set up a pipeline to extract and prepare raw data for the model

## Setup and Utils

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets.samples_generator import make_classification
from sklearn.feature_selection import VarianceThreshold
from sklearn.pipeline import Pipeline

## Generate fake data

In [2]:
def make_data():
    
    X, y = make_classification(weights=[0.7])

    # zero variance column
    zero_col = np.zeros(shape=(100, 1))

    # categorical column
    cat_col = ['male'] * 20 + ['Male'] * 20 + ['FEMALE'] * 30 + ['CORGI'] * 15 + ['others'] * 15
             
    df = pd.DataFrame(data=X,
                      columns=[f'col_{i}' for i in range(20)])
    df['zero_col'] = zero_col
    df['cat_col'] = cat_col
    
    print('Sample dim: ', df.shape)
    print('Lable proportion: ', np.bincount(y))

    return df, y
    
    

In [3]:
features, label = make_data()

Sample dim:  (100, 22)
Lable proportion:  [70 30]


## Clean up the data

- reduce data size by change data types i.e. float64 -> float32
- filtering out data early
- data values that are equivalent - i.e. 'Male' vs 'male'
- remove wrong entries
- avoid spaces in your features name 
- one-hot-encoding

In [4]:
print(features['cat_col'].unique())

['male' 'Male' 'FEMALE' 'CORGI' 'others']


In [5]:
# lower case
features['cat_col'] = features['cat_col'].str.lower()
print(features['cat_col'].unique())

['male' 'female' 'corgi' 'others']


In [6]:
# remove wrong entries
features_filter = features.query('cat_col != "corgi"')

In [7]:
features_dummy = pd.get_dummies(features_filter, prefix='gender', columns=['cat_col'])

## Drop useless features

- Features that you know is not relevant to your problem
- A feature that is linear combination of other features
- Features that has little variance

In [8]:
try: 
    features_dummy.drop('gender_others', axis=1, inplace=True)
except KeyError:
    pass

In [9]:
def drop_zero_var_col(df):
    selector = VarianceThreshold()
    selector.fit(df)
    non_zero_var_col_mask = selector.get_support(indices=False)
    
    return df.loc[:, non_zero_var_col_mask]

In [10]:
final_features = drop_zero_var_col(features_dummy).shape

## Missing values

- Find out the reason that is missing
- Drop them if not a log
- Common method:
    - A constant value i.e. -99
    - mean, median, mode
    - statistical models
- You don't know which one is good until you evaluate your model

## Create your training and testing set (train-test-split)

![](https://miro.medium.com/max/764/0*46cSU6wIIOztcY5P.)

Something to keep in mind

- Always set a seed so you can compare
- Test size is traditionally 80/20 
- Do not shuffle if you are dealing with time series
- Stratification is essential when you are dealing with unbalanced labels

Common Mistakes (data leakage)
- Time Series: Incorporating information from the future in your model (i.e., quarterly results before end-of-quarter)
- Imputing based on the combined train-test dataset
- Taking a peek on the test data halfway through training your model

In [11]:
(train_features, test_features,
 train_label, test_label) = train_test_split(
    features,
    label,
    random_state=613,
    shuffle=True,
    test_size=0.20, # This is fine enough
    stratify=label
)

In [12]:
# save your training test set for future reference and usage
# train_features.to_csv('train_X.csv'); test_features.to_csv('test_X.csv')
# train_label.to_csv('train_y.csv'); test_label.to_csv('test_X.csv')

## Others?

- Use [sklearn.pipeline.Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline) to make a Pipeline from transforming data to training model
- Use [cross-Validation](https://scikit-learn.org/stable/modules/cross_validation.html) when you have a small data set (or even you have a huge data set)
- [Data normlization](https://scikit-learn.org/stable/modules/preprocessing.html)
- Outliers...

## Summary

- Adjust data type & fix erroneous values
- Select only features that are relevant
- Remove data with (near-) 0 variance
- One-hot-encoding for categorical variable
- Always split your data into train/test 

## Bonus: Tips, tricks

- there are many ways to do the same thing in Pandas
- Some better, some worse, some equvilaent. But again, there are a lot.....
- Let's standardize some of them to make our life easier

### Select rows and columns

- we have some options here: dot notation, .iloc, .loc, \[](\_\_getitem__)
- avoid dot notation
- Prefer `.iloc[]` and `.loc[]`
- For performance: query()

In [13]:
def generate_data(n_row=1000, n_num_col=10):
    '''Generate fake data
    Args:
        n_row(int): number of rows for the data
        n_num_col(int): number of numeric columns
    
    Returns:
        pandas.DataFrame with the allocated size.
    '''
    if (n_row <= 0 and n_num_col <= 0):
        raise ValueError('Check your dimension. should be both integers larger than zero')
        
    cols = [f'col_{i}' for i in range(n_num_col)]
    output = pd.DataFrame(
        data=np.random.randn(n_row, n_num_col).astype('float32'),
        columns=cols)
    return output

In [14]:
small_data = generate_data(int(1e6), 10)
small_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 10 columns):
col_0    1000000 non-null float32
col_1    1000000 non-null float32
col_2    1000000 non-null float32
col_3    1000000 non-null float32
col_4    1000000 non-null float32
col_5    1000000 non-null float32
col_6    1000000 non-null float32
col_7    1000000 non-null float32
col_8    1000000 non-null float32
col_9    1000000 non-null float32
dtypes: float32(10)
memory usage: 38.1 MB


This data is aroudn ~38.1 MB

### Select only one column
- avoid dot notation
    - it cannot be used when there is a space in the column (i.e. 'col 1')
    - may get into conflicts with object methods (i.e. column 'loc' for location)
    - hard to read
- Prefer .loc for consitency and readbility
- df['col'] is okay..

In [15]:
# bad
# small_data.col_1

# good
assert(small_data['col_1'] is small_data.loc[:, 'col_1'])

#### Slicing rows and columns
- `.iloc()` does not include the right bound 
- `.loc()`  include the right bound

In [16]:
assert(small_data.iloc[:1000, :5]                 # position indexing
       .equals(
       small_data.loc[:999, 'col_0':'col_4']      # lable indexing
       )
)


### .query() for performance

- Use [numepxr](https://github.com/pydata/numexpr) under the hood.
- multi-threaded, less memory, use VML 

Here cannot really see the difference. But depends on the operation, it will be a huge performance gain.

In [17]:
%timeit small_data.query('col_1 > 0 & col_2 > 0')

12.4 ms ± 169 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [18]:
%timeit small_data.loc[(small_data['col_1'] > 0) & (small_data['col_2'] > 0)]

14.7 ms ± 293 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## NA related methods: `isna` and `isnull`

- we have `isna`, `isnull`, `notna`, `notnull`
- Others: `dropna` and `fillna`.
- `*na` and `*null` are the same, they are alias
- use `*na` only to match the other `na` methods
- Source https://github.com/pandas-dev/pandas/blob/cb00deb94500205fcb27a33cc1d0df79a9727f8b/pandas/core/dtypes/missing.py#L102


## Best way to loop over a dataframe

- For details and benchmark, please refer to [A Beginner’s Guide to Optimizing Pandas Code for Speed](https://engineering.upside.com/a-beginners-guide-to-optimizing-pandas-code-for-speed-c09ef2c6a4d6)

#### TL;DR - from the slowest to the fastest

1. Crude looping over DataFrame rows using indices `for ...`
2. Looping with iterrows()
3. Looping with apply()
4. Vectorization with Pandas series
5. Vectorization with NumPy arrays